In [1]:
from model import *
import tensorflow as tf
import pandas as pd

2023-05-19 08:26:04.975790: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
fe = tf.keras.models.load_model('feature_extractor.h5')
fe.summary()

In [ ]:
fe = get_feature_extractor('models/aug_3.hdf5')
fe.save('feature_extractor.h5')

In [2]:
dual_model = dual_net((224, 224, 1), 15, 1)
dual_model.load_weights('models/dual_model_4.hdf5')
losses = {
    "age": "categorical_crossentropy",
    "gender": "binary_crossentropy"
}
loss_weights = {"age": 0.99, "gender": 0.01}
metrics=['accuracy',
         tf.keras.metrics.TopKCategoricalAccuracy(),
         tf.keras.metrics.Precision(),
         tf.keras.metrics.Recall()]
dual_model.compile(optimizer=Adam(lr=0.0001),
    loss=losses,
    loss_weights=loss_weights,
    metrics=metrics)

dual_model.save('dual_model.h5')

2023-05-19 08:26:09.535726: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
# plot the model using keras plot_model
tf.keras.utils.plot_model(model2, to_file='model2.png', show_shapes=True, show_layer_names=True)

In [3]:
single_model = single_net((224, 224, 1), 30)
single_model.load_weights('models/aug_3.hdf5')
single_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=metrics)
single_model.save('single_model.h5')

In [ ]:
# plot the model using keras plot_model
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
input = Input(shape=(224, 224, 1))

# base model
conv1 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(input)
conv1 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv1)
mpool1 = MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv1)

conv2 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(mpool1)
conv2 = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv2)
mpool2 = MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv2)

conv3 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(mpool2)
conv3 = Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv3)
mpool3 = MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(mpool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer="he_normal", bias_initializer="zeros")(conv4)
mpool4 = MaxPooling2D(pool_size = (2,2), strides = (2,2))(conv4)

model = Model(inputs=input, outputs=mpool4)

In [ ]:
model.summary(line_length=100)

In [ ]:
table=pd.DataFrame(columns=["Name","Type","Shape"])
for layer in model.layers:
    table = table.append({"Name":layer.name, "Type": layer.__class__.__name__,"Shape":layer.output_shape}, ignore_index=True)